<a href="https://colab.research.google.com/github/y-ashaswini/notebooks/blob/main/fine_tuning_problem_statement_3_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch scikit-learn matplotlib peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import pandas as pd
import time

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
# Load Sentiment140 dataset
url = "/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv"
sentiment_df = pd.read_csv(url, encoding='latin-1', header=None)
sentiment_df.columns = ['polarity', 'id', 'date', 'query', 'user', 'text']

# Map labels to 0 and 1 (assuming 0 is negative, 4 is positive in Sentiment140 dataset)
sentiment_df['label'] = sentiment_df['polarity'].map({0: 0, 4: 1})

# Only keep necessary columns
sentiment_df = sentiment_df[['text', 'label']].dropna()

# Check the size of the dataset
print(f"Dataset size: {len(sentiment_df)}")

# Reduce dataset size for quicker training if the dataset is large
if len(sentiment_df) > 10000:
    sentiment_df = sentiment_df.sample(n=10000, random_state=42)

dataset = Dataset.from_pandas(sentiment_df)
dataset = dataset.train_test_split(test_size=0.2)
dataset

In [ ]:
# Define tokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Function to tokenize data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
# Metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)  # Convert logits to tensor
    acc = accuracy_score(labels, predictions)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec}

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
# Function to train and evaluate model
def train_and_evaluate(model, training_args, tokenized_datasets):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        compute_metrics=compute_metrics,
    )

    start_time = time.time()
    trainer.train()
    train_time = time.time() - start_time

    start_time = time.time()
    eval_results = trainer.evaluate()
    eval_time = time.time() - start_time

    eval_results['train_time'] = train_time
    eval_results['eval_time'] = eval_time

    return eval_results

In [ ]:
# Base Model without Soft Prompts or LoRA
base_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
base_results = train_and_evaluate(base_model, training_args, tokenized_datasets)
print("Base Model Results:", base_results)

In [ ]:
import torch.nn as nn

class SoftPromptModel(AutoModelForSequenceClassification):
    def __init__(self, config, num_labels, soft_prompt_length=20):
        super().__init__(config)
        self.num_labels = num_labels
        self.soft_prompt_length = soft_prompt_length
        self.soft_prompt_embed = nn.Embedding(self.soft_prompt_length, config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size, num_labels)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        soft_prompt=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

In [ ]:
soft_prompt_model = SoftPromptModel.from_pretrained(model_checkpoint, num_labels=2)
soft_prompt_results = train_and_evaluate(soft_prompt_model, training_args, tokenized_datasets)
print("Soft Prompt Results:", soft_prompt_results)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification task
    r=8,                        # Low rank
    lora_alpha=32,              # Alpha parameter for LoRA
    lora_dropout=0.1,           # Dropout for LoRA
    bias="none",
    target_modules=["q_lin", "v_lin"]
)

# Apply LoRA to the base model
lora_model = get_peft_model(AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2), lora_config)

# Train and evaluate the LoRA model
lora_results = train_and_evaluate(lora_model, training_args, tokenized_datasets)
print("LoRA Model Results:", lora_results)

In [ ]:
# Visualization
labels = ["Base Model", "Soft Prompts", "LoRA"]
accuracy = [base_results.get("eval_accuracy", 0), soft_prompt_results.get("eval_accuracy", 0), lora_results.get("eval_accuracy", 0)]
f1 = [base_results.get("eval_f1", 0), soft_prompt_results.get("eval_f1", 0), lora_results.get("eval_f1", 0)]
precision = [base_results.get("eval_precision", 0), soft_prompt_results.get("eval_precision", 0), lora_results.get("eval_precision", 0)]
recall = [base_results.get("eval_recall", 0), soft_prompt_results.get("eval_recall", 0), lora_results.get("eval_recall", 0)]
train_time = [base_results.get("train_time", 0), soft_prompt_results.get("train_time", 0), lora_results.get("train_time", 0)]
eval_time = [base_results.get("eval_time", 0), soft_prompt_results.get("eval_time", 0), lora_results.get("eval_time", 0)]

fig, axs = plt.subplots(3, 2, figsize=(15, 15))
axs[0, 0].bar(labels, accuracy, color='b')
axs[0, 0].set_title('Accuracy')
axs[0, 1].bar(labels, f1, color='g')
axs[0, 1].set_title('F1 Score')
axs[1, 0].bar(labels, precision, color='r')
axs[1, 0].set_title('Precision')
axs[1, 1].bar(labels, recall, color='c')
axs[1, 1].set_title('Recall')
axs[2, 0].bar(labels, train_time, color='m')
axs[2, 0].set_title('Training Time')
axs[2, 1].bar(labels, eval_time, color='y')
axs[2, 1].set_title('Evaluation Time')

for ax in axs.flat:
    ax.set_ylabel('Score')
    ax.set_xlabel('Model Type')
    ax.label_outer()

plt.tight_layout()
plt.show()